# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
 # Data Science
import pandas as pd
import numpy as np

# API Requests
from pprint import pprint
import requests
import json

# Data Viz
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Port-aux-Français,-49.3500,70.2167,43.03,95.0,96.0,16.69,TF,1.735933e+09
1,1,Grytviken,-54.2811,-36.5092,48.27,82.0,100.0,10.07,GS,1.735933e+09
2,2,Blackmans Bay,-43.0167,147.3167,57.33,91.0,75.0,3.89,AU,1.735933e+09
3,3,Griffith,41.5284,-87.4237,25.84,62.0,75.0,11.50,US,1.735933e+09
4,4,Cabo San Lucas,22.8909,-109.9124,79.11,50.0,0.0,6.91,MX,1.735933e+09


In [11]:
# Date Cleaning w/dates
city_data_df["Date"] = pd.to_datetime(city_data_df.Date * 1e9)
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Port-aux-Français,-49.3500,70.2167,43.03,95.0,96.0,16.69,TF,2025-01-03 19:36:33
1,1,Grytviken,-54.2811,-36.5092,48.27,82.0,100.0,10.07,GS,2025-01-03 19:36:34
2,2,Blackmans Bay,-43.0167,147.3167,57.33,91.0,75.0,3.89,AU,2025-01-03 19:36:35
3,3,Griffith,41.5284,-87.4237,25.84,62.0,75.0,11.50,US,2025-01-03 19:31:48
4,4,Cabo San Lucas,22.8909,-109.9124,79.11,50.0,0.0,6.91,MX,2025-01-03 19:36:37


In [12]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   City_ID     624 non-null    int64         
 1   City        594 non-null    object        
 2   Lat         594 non-null    float64       
 3   Lng         594 non-null    float64       
 4   Max Temp    594 non-null    float64       
 5   Humidity    594 non-null    float64       
 6   Cloudiness  594 non-null    float64       
 7   Wind Speed  594 non-null    float64       
 8   Country     590 non-null    object        
 9   Date        594 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(6), int64(1), object(2)
memory usage: 48.9+ KB


In [13]:
# Drop rows with any null values
city_data_df = city_data_df.dropna()

city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 590 entries, 0 to 623
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   City_ID     590 non-null    int64         
 1   City        590 non-null    object        
 2   Lat         590 non-null    float64       
 3   Lng         590 non-null    float64       
 4   Max Temp    590 non-null    float64       
 5   Humidity    590 non-null    float64       
 6   Cloudiness  590 non-null    float64       
 7   Wind Speed  590 non-null    float64       
 8   Country     590 non-null    object        
 9   Date        590 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(6), int64(1), object(2)
memory usage: 50.7+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols=["City", "Country", "Max Temp"]  # Add 'city' to the tooltip
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities = city_data_df[
    (city_data_df["Max Temp"] >= 15) & 
    (city_data_df["Max Temp"] <= 35) &
    (city_data_df["Wind Speed"] < 15) &
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_cities = ideal_cities.dropna()

# Display sample data
ideal_cities.info()
ideal_cities.head()

<class 'pandas.core.frame.DataFrame'>
Index: 12 entries, 40 to 549
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   City_ID     12 non-null     int64         
 1   City        12 non-null     object        
 2   Lat         12 non-null     float64       
 3   Lng         12 non-null     float64       
 4   Max Temp    12 non-null     float64       
 5   Humidity    12 non-null     float64       
 6   Cloudiness  12 non-null     float64       
 7   Wind Speed  12 non-null     float64       
 8   Country     12 non-null     object        
 9   Date        12 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(6), int64(1), object(2)
memory usage: 1.0+ KB


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
40,40,Stanley,54.8680,-1.6985,27.52,95.0,0.0,1.14,GB,2025-01-03 19:37:17
145,145,Fengcheng,40.4536,124.0717,18.54,42.0,0.0,0.20,CN,2025-01-03 19:39:14
157,157,Arafat,39.0344,44.6198,24.31,44.0,0.0,4.72,IR,2025-01-03 19:39:27
230,230,Beipiao,41.7919,120.7792,18.39,37.0,0.0,1.57,CN,2025-01-03 19:40:50
295,295,Yatou,37.1500,122.3833,26.55,45.0,0.0,3.13,CN,2025-01-03 19:42:02


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
40,Stanley,GB,54.8680,-1.6985,95.0,
145,Fengcheng,CN,40.4536,124.0717,42.0,
157,Arafat,IR,39.0344,44.6198,44.0,
230,Beipiao,CN,41.7919,120.7792,37.0,
295,Yatou,CN,37.1500,122.3833,45.0,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",  # Search for hotels
    "apiKey": geoapify_key,  # Your Geoapify API key
    "limit": 1  # Limit the response to the closest hotel
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set the base URL for Geoapify Places API
    base_url = "https://api.geoapify.com/v2/places"

    # Make the API request
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Try to extract the hotel name from the response
    try:
        hotel_name = name_address["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search result
    print(f"{row['City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Stanley - nearest hotel: Hotel 52
Fengcheng - nearest hotel: 鸿金门大酒店
Arafat - nearest hotel: No hotel found
Beipiao - nearest hotel: No hotel found
Yatou - nearest hotel: 盛悦商务酒店
Xigazê - nearest hotel: Gesar Hotel
Kofu - nearest hotel: ホテル談露館
Bridlington - nearest hotel: Brunswick Hotel
Kaifeng - nearest hotel: JinjiangInn(Kaifeng)
Kujang-ŭp - nearest hotel: No hotel found
Zhuanghe - nearest hotel: 非繁城品酒店
Arrondissement du Havre - nearest hotel: Le Fécamp


,City,Country,Lat,Lng,Humidity,Hotel Name
40,Stanley,GB,54.8680,-1.6985,95.0,Hotel 52
145,Fengcheng,CN,40.4536,124.0717,42.0,鸿金门大酒店
157,Arafat,IR,39.0344,44.6198,44.0,No hotel found
230,Beipiao,CN,41.7919,120.7792,37.0,No hotel found
295,Yatou,CN,37.1500,122.3833,45.0,盛悦商务酒店
306,Xigazê,CN,29.2500,88.8833,28.0,Gesar Hotel
319,Kofu,JP,35.6639,138.5683,73.0,ホテル談露館
334,Bridlington,GB,54.0831,-0.1919,72.0,Brunswick Hotel
359,Kaifeng,CN,34.7911,114.3483,54.0,JinjiangInn(Kaifeng)
486,Kujang-ŭp,KP,39.8672,126.0303,46.0,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x="Lng",  # Longitude
    y="Lat",  # Latitude
    geo=True,  # Enable geographic plotting
    tiles="CartoLight",  # Use a light map tile
    frame_width=700,  # Adjust map width
    frame_height=500,  # Adjust map height
    size=10,  # Set marker size
    color="Humidity",  # Color points by Humidity
    hover_cols=["City", "Country", "Hotel Name", "Humidity"]  # Add additional hover information
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)